In [1]:
%matplotlib auto
import numpy as np
import matplotlib.pyplot as plt
import csv

def c_decode(c_str):
    c_coeff = []
    for i in c_str:
        if i == "2":
            c_coeff.append(-1)
        if i == "0":
            c_coeff.append(0)
        if i == "1":
            c_coeff.append(1)
    return c_coeff

# compute prob of tamplate
def prob(n, cov_mat, N):
    det = np.linalg.det(cov_mat)
    cov_mat_inv = np.linalg.pinv(cov_mat)
    return 1/np.sqrt(det * pow(2*np.pi,N))*np.exp(-0.5*np.dot(np.dot(n.T,cov_mat_inv), n))

def add_noise(signal, snr_db):
    signal_power = np.mean(np.abs(signal) ** 2)
    snr = 10 ** (snr_db / 10)
    noise_power = signal_power / snr
    noise = np.sqrt(noise_power) * np.random.randn(*signal.shape)
    noisy_signal = signal + noise 
    return noisy_signal

def tamplate_build(POI_lst:list, profiled_trc:np.array, SNR):

    POI_num = len(POI_lst)
    tamplate_num = len(profiled_trc)
    profiled_trc_num = len(profiled_trc[0])
    profiled_trc_POI = np.zeros([tamplate_num, profiled_trc_num, POI_num])
    for i in range(0, tamplate_num):
        for j in range(0, profiled_trc_num):
            trc = add_noise(profiled_trc[i][j], SNR)
            for k in range(0, POI_num):
                profiled_trc_POI[i][j][k] = trc[POI_lst[k]]

    # build tamplates for load s[i]
    # compute average trace Mi for each tamplate
    profiled_trc_POI_mean = np.zeros([tamplate_num, POI_num])
    for i in range(0, tamplate_num):
        profiled_trc_POI_mean[i] = np.mean(profiled_trc_POI[i], axis=0)

    # compute noise vector for all L sample traces
    noise_vec = np.zeros([tamplate_num, profiled_trc_num, POI_num])
    for i in range(0, tamplate_num):
        for j in range(0, profiled_trc_num):
            for k in range(0, POI_num):
                noise_vec[i][j][k] =  profiled_trc_POI[i][j][k] - profiled_trc_POI_mean[i][k]

    # compute cov matrix of corresponding noise vector
    cov_mat = np.zeros([tamplate_num, POI_num, POI_num])
    for i in range(0, tamplate_num):
        cov_mat[i] = np.cov(noise_vec[i], rowvar=False)
    
    return [profiled_trc_POI_mean, cov_mat]

def tamplate_match(POI_lst:list, trc:np.array, hw_tamplate:list, SNR):
    profiled_trc_mean, cov_mat = hw_tamplate
    POI_num = len(POI_lst)
    test_trc_num = len(trc)
    test_trc = np.zeros([test_trc_num, POI_num])
    for i in range(0, test_trc_num):
        trc_nv = add_noise(trc[i], SNR)
        for k in range(0, POI_num):
            test_trc[i][k] = trc_nv[POI_lst[k]]

    # compute the probability of test traces
    tamplate_num = len(profiled_trc_mean)
    poss_value_lst = np.zeros([test_trc_num, tamplate_num])
    for k in range(0, test_trc_num):
        nv = np.zeros([tamplate_num, POI_num])
        for i in range(0, tamplate_num):
            for j in range(0, POI_num):
                nv[i][j] = test_trc[k][j] - profiled_trc_mean[i][j]
            poss_value_lst[k][i] = prob(nv[i], cov_mat[i], POI_num)

    # sort Hamming weights according to probability
    predict_hw_lst = np.zeros([test_trc_num, tamplate_num])
    predict_prob_lst = np.zeros([test_trc_num, tamplate_num])
    for k in range(0, test_trc_num):
        val = poss_value_lst[k]
        hw = np.array(list(range(0, tamplate_num))) 
        index = np.argsort(val)
        predict_hw_lst[k] = hw[index]
        predict_prob_lst[k] = val[index]
    
    return [predict_hw_lst, predict_prob_lst]


Using matplotlib backend: TkAgg


In [2]:
# read test data from s

# coeffs of test data
c_lst_test = []
for line in open("test_trc/challenge_c.txt"):
    c_lst_test.append(c_decode(line[:-1]))

# traces of test data
trc_len = 39085
test_trc_lst = np.zeros([len(c_lst_test), trc_len])
for i in range(0, len(c_lst_test)):
    test_trc_lst[i] = np.loadtxt(f"test_trc/test_trc{i+1}.trc")


In [3]:
trc_len = 39085
# coeffs of load a & b
c_lst_ldr_ab = []
for line in open("ab_tamplate/challenge_c.txt"):
    c_lst_ldr_ab.append(line[:-1])

# traces of load a & b
profiled_lst_ldr_ab = np.zeros([len(c_lst_ldr_ab), trc_len])
for i in range(0, len(c_lst_ldr_ab)):
    profiled_lst_ldr_ab[i] = np.loadtxt(f"ab_tamplate/trc{i}.trc")

In [216]:
SNR = 100
ave_len = 305
st = 21

In [217]:
tamplate_num = 3 
profiled_trc_num = 1536
POI_lst = [227, 268, 270]
cnt = [0]*tamplate_num


Sample_ldr_a = np.zeros([tamplate_num, profiled_trc_num, ave_len]) # sample traces for {-1,0,1}
for j in range(0, len(c_lst_ldr_ab)):
   c_ans = c_decode(c_lst_ldr_ab[j])
   for i in range(0, 128):
        idc = c_ans[i]+1
        Sample_ldr_a[idc][cnt[idc]] = profiled_lst_ldr_ab[j][st+i*ave_len:st+(i+1)*ave_len]
        cnt[idc] += 1

tamplate_ldr_a = tamplate_build(POI_lst, Sample_ldr_a, SNR)

# test tamplate of load a
ave_success_rate = 0
for q in range(0, len(c_lst_test)):
    trc_blk= np.zeros([128, ave_len])
    for i in range(0, 128):
        trc_blk[i] = test_trc_lst[q][st+i*ave_len:st+(i+1)*ave_len]

    [predict_hw_ldra_lst, predict_prob_ldra_lst] = tamplate_match(POI_lst, trc_blk, tamplate_ldr_a, SNR)

    success_rate = 0
    for i in range(0, 128):
        if predict_hw_ldra_lst[i][-1] == c_lst_test[q][i] + 1:
            success_rate += 1
    # print(f"trace {q+1}: success rate =",success_rate/128)
    ave_success_rate += success_rate
print("ave success rate = ", ave_success_rate/(128*len(c_lst_test)))

ave success rate =  1.0


In [218]:
# tamplate for b
profiled_trc_num = 1536
tamplate_num = 3 
POI_lst = [51,163,183]
cnt = [0]*tamplate_num

Sample_ldr_b = np.zeros([tamplate_num, profiled_trc_num, ave_len]) # sample traces for {-1,0,1}
for j in range(0, len(c_lst_ldr_ab)):
   c_ans = c_decode(c_lst_ldr_ab[j])[128:]
   for i in range(0, 128):
        idc = c_ans[i]+1
        Sample_ldr_b[idc][cnt[idc]] = profiled_lst_ldr_ab[j][st+i*ave_len:st+(i+1)*ave_len]
        cnt[idc] += 1

tamplate_ldr_b = tamplate_build(POI_lst, Sample_ldr_b, SNR)

# test tamplate of load a
ave_success_rate = 0
for q in range(0, len(c_lst_test)):
    trc_blk= np.zeros([128, ave_len])
    for i in range(0, 128):
        trc_blk[i] = test_trc_lst[q][st+i*ave_len:st+(i+1)*ave_len]

    [predict_hw_ldrb_lst, predict_prob_ldrb_lst] = tamplate_match(POI_lst, trc_blk, tamplate_ldr_b, SNR)

    success_rate = 0
    for i in range(0, 128):
        if predict_hw_ldrb_lst[i][-1] == c_lst_test[q][i+128] + 1:
            success_rate += 1
    # print(f"trace {q+1}: success rate =",success_rate/128)
    ave_success_rate += success_rate
print("ave success rate = ", ave_success_rate/(128*len(c_lst_test)))

ave success rate =  1.0


In [219]:
# build tamplate for store b
trc_len = 39085
c_lst_str_b = []
for line in open("hwb_tamplate/challenge_c.txt"):
    c_lst_str_b.append(line[1:-3])

# hwb trc of profiled c
profiled_lst_str_b = np.zeros([len(c_lst_str_b), trc_len])
for i in range(0, len(c_lst_str_b)):
    profiled_lst_str_b[i] = np.loadtxt(f"hwb_tamplate/hwb{i+1}.trc")

hwb_set = [0,1,6,13,14,18,19,32]
hw_b = [[5,7,3], [6,0,4], [6,1,2]]
profiled_trc_num = 1536 # sample number
tamplate_num = 8 # number of tamplates
POI_lst = [250] # POI
cnt = [0]*tamplate_num

# assign trace blks into corresponding Sample list
Sample_str_b = np.zeros([tamplate_num, profiled_trc_num, ave_len]) # according array for sample traces
for j in range(0, len(c_lst_str_b)):
   c_ans = c_decode(c_lst_str_b[j])
   for i in range(0, 128):
        idc = hw_b[c_ans[i]+1][c_ans[i+128]+1]
        Sample_str_b[idc][cnt[idc]] = profiled_lst_str_b[j][st+i*ave_len:st+(i+1)*ave_len]
        cnt[idc] += 1  

for k in range(0, tamplate_num):
    assert(cnt[k] == profiled_trc_num)

tamplate_str_b = tamplate_build(POI_lst, Sample_str_b, SNR)

# test tamplate of store b
ave_HW_1 = 0
ave_HW_5 = 0
HW1_num = np.zeros(tamplate_num)
HW5_num = np.zeros(tamplate_num)
cnt_num = np.zeros(tamplate_num)
for q in range(0, len(c_lst_test)):
    trc_blk= np.zeros([128, ave_len])
    for i in range(0,128):
        trc_blk[i] = test_trc_lst[q][st+i*ave_len:st+(i+1)*ave_len]

    [predict_hw_strb_lst, predict_prob_strb_lst] = tamplate_match(POI_lst, trc_blk, tamplate_str_b, SNR)
    
    HW_1 = 0
    HW_5 = 0
    ct_ans = c_lst_test[q]
    for i in range(0, 128):
        hw = hwb_set[hw_b[ct_ans[i]+1][ct_ans[i+128]+1]]
        cnt_num[hw_b[ct_ans[i]+1][ct_ans[i+128]+1]]+=1
        res = [hwb_set[int(t)] for t in predict_hw_strb_lst[i][-5:]]
        if res[-1] == hw:
            HW_1 += 1
            HW1_num[hw_b[ct_ans[i]+1][ct_ans[i+128]+1]]+=1
        if hw in res:
            HW_5 += 1
            HW5_num[hw_b[ct_ans[i]+1][ct_ans[i+128]+1]]+=1
    # print(f"trace {q}: HW 1 rate =",HW_1/128)
    # print(f"trace {q}: HW 5 rate =",HW_5/128)
    ave_HW_1 += HW_1
    ave_HW_5 += HW_5
print("ave HW 1 = ", ave_HW_1/(128*len(c_lst_test)))
print("ave HW 5 = ", ave_HW_5/(128*len(c_lst_test)))

ave HW 1 =  1.0
ave HW 5 =  1.0


In [220]:
# build tamplate for store a
trc_len = 39085
c_lst_str_a = []
for line in open("hwa_tamplate/challenge_c.txt"):
    c_lst_str_a.append(line[1:-3])

# hwa trc of profiled c
profiled_lst_str_a = np.zeros([len(c_lst_str_a), trc_len])
for i in range(0, len(c_lst_str_a)):
    profiled_lst_str_a[i] = np.loadtxt(f"hwa_tamplate/hwa{i+1}.trc")

hwa_set = [0,1,6,13,14,18,191,192,32]
hw_a = [[3,8,5], [4,0,6], [2,1,7]]
profiled_trc_num = 1536 # sample number
tamplate_num = 9 # number of tamplates
POI_lst = [292] # POI
cnt = [0]*tamplate_num
# assign trace blks into corresponding Sample list
Sample_str_a = np.zeros([tamplate_num, profiled_trc_num, ave_len]) # according array for sample traces
for j in range(0, len(c_lst_str_a)):
   c_ans = c_decode(c_lst_str_a[j])
   for i in range(0, 128):
        idc = hw_a[c_ans[i]+1][c_ans[i+128]+1]
        Sample_str_a[idc][cnt[idc]] = profiled_lst_str_a[j][st+i*ave_len:st+(i+1)*ave_len]
        cnt[idc] += 1  

for k in range(0, tamplate_num):
    assert(cnt[k] == profiled_trc_num)

tamplate_str_a = tamplate_build(POI_lst, Sample_str_a, SNR)

# test tamplate of store a
ave_HW_1 = 0
ave_HW_5 = 0
HW1_num = np.zeros(tamplate_num)
HW5_num = np.zeros(tamplate_num)
cnt_num = np.zeros(tamplate_num)
for q in range(0, len(c_lst_test)):
    trc_blk = np.zeros([128, ave_len])
    for i in range(0,128):
        trc_blk[i] = test_trc_lst[q][st+i*ave_len:st+(i+1)*ave_len]

    [predict_hw_stra_lst, predict_prob_stra_lst] = tamplate_match(POI_lst, trc_blk, tamplate_str_a, SNR)
    
    HW_1 = 0
    HW_5 = 0
    ct_ans = c_lst_test[q]
    for i in range(0, 128):
        hw = hwa_set[hw_a[ct_ans[i]+1][ct_ans[i+128]+1]]
        cnt_num[hw_a[ct_ans[i]+1][ct_ans[i+128]+1]]+=1
        res = [hwa_set[int(t)] for t in predict_hw_stra_lst[i][-5:]]
        if res[-1] == hw:
            HW_1 += 1
            HW1_num[hw_a[ct_ans[i]+1][ct_ans[i+128]+1]]+=1
        if hw in res:
            HW_5 += 1
            HW5_num[hw_a[ct_ans[i]+1][ct_ans[i+128]+1]]+=1
    # print(f"trace {q}: HW 1 rate =",HW_1/128)
    # print(f"trace {q}: HW 5 rate =",HW_5/128)
    ave_HW_1 += HW_1
    ave_HW_5 += HW_5
print("ave HW 1 = ", ave_HW_1/(128*len(c_lst_test)))
print("ave HW 5 = ", ave_HW_5/(128*len(c_lst_test)))
'''
for k in range(0, tamplate_num):
    print(f"tamplate{hwa_set[k]}:")
    print("ave HW1 = ", HW1_num[k]/cnt_num[k])
    print("ave HW5 = ", HW5_num[k]/cnt_num[k])
'''

ave HW 1 =  1.0
ave HW 5 =  1.0


'\nfor k in range(0, tamplate_num):\n    print(f"tamplate{hwa_set[k]}:")\n    print("ave HW1 = ", HW1_num[k]/cnt_num[k])\n    print("ave HW5 = ", HW5_num[k]/cnt_num[k])\n'

In [221]:
def dfs(pos, c, res_a, res_b):
    if pos == 128:
        cnt = 0
        for i in c:
            if i in [-1, 1]:
                cnt+=1
        if cnt == 39:
            return [True, c]
        else:
            return [False, None]
        
    cnt = 0
    for i in range(0, pos):
        if c[i] in [-1,1]:
            cnt+=1
        if c[i+128] in [-1,1]:
            cnt+=1
    if cnt>39:
        return [False, None]
    
    for i in range(0, len(res_a[pos])):
        p = -(i+1)
        c[pos] = res_a[pos][p]
        c[pos+128] = res_b[pos][p]
        [F, rc] = dfs(pos+1, c, res_a, res_b)
        if F:
            return [True, rc]
        
    return [False, 'None correct']
        
def search_alg(ldra_res, ldrb_res, stra_res, strb_res):
    hw_ldra_lst, prob_ldra_lst = ldra_res
    hw_ldrb_lst, prob_ldrb_lst = ldrb_res
    hw_stra_lst, prob_stra_lst = stra_res
    hw_strb_lst, prob_strb_lst = strb_res
    
    res_a = []
    res_b = []
    res_p = []
    for q in range(0, 128):
        hw_stra = [hwa_set[int(t)] for t in hw_stra_lst[q][-5:]]
        prob_stra = prob_stra_lst[q][-5:]
        hw_strb = [hwb_set[int(t)] for t in hw_strb_lst[q][-5:]]
        prob_strb = prob_strb_lst[q][-5:]
        final_a_lst = []
        final_b_lst = []
        final_prob = []
        for ila in range(0, 3):
            la = int(hw_ldra_lst[q][ila])
            for ilb in range(0, 3):
                lb = int(hw_ldrb_lst[q][ilb])
                sa = hwa_set[hw_a[la][lb]]
                sb = hwb_set[hw_b[la][lb]] 
                if sa not in hw_stra or sb not in hw_strb:
                    continue
                isa = hw_stra.index(sa)
                isb = hw_strb.index(sb)
                p = prob_ldra_lst[q][ila]*prob_ldrb_lst[q][ilb]*prob_stra[isa]*prob_strb[isb]
                final_a_lst.append(la-1)
                final_b_lst.append(lb-1)
                final_prob.append(p)
        
        final_a_lst = np.array(final_a_lst) 
        final_b_lst = np.array(final_b_lst)
        final_prob = np.array(final_prob)
        ind = np.argsort(final_prob)
        final_a_lst = final_a_lst[ind]
        final_b_lst = final_b_lst[ind]
        final_prob = final_prob[ind]
        t = final_prob > 1e-29
        res_a.append(final_a_lst[t].tolist())
        res_b.append(final_b_lst[t].tolist())
        res_p.append(final_prob[t].tolist())
        if len(res_p[-1]) == 0:
            return [False, 'Empty list']
    
    c = [0]*256
    print('search begin')
    return dfs(0, c, res_a, res_b)
    
    


In [222]:
q = 2
trc_blk = np.zeros([128, ave_len])
for i in range(0,128):
    trc_blk[i] = test_trc_lst[q][st+i*ave_len:st+(i+1)*ave_len]

ldra_res = tamplate_match([227, 268, 270], trc_blk, tamplate_ldr_a, SNR)
ldrb_res = tamplate_match([51,163,183], trc_blk, tamplate_ldr_b, SNR)
stra_res = tamplate_match([292], trc_blk, tamplate_str_a, SNR)
strb_res = tamplate_match([250], trc_blk, tamplate_str_b, SNR)

T, rc = search_alg(ldra_res, ldrb_res, stra_res, strb_res)
print(f'search res: {T}')
print(rc)
print(f'recover res validation : {c_lst_test[q] == rc}')


search begin
search res: True
[0, 0, 0, 0, -1, 0, 1, 0, -1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, -1, 0, 0, -1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, -1, 0, 0, -1, 1, 1, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0]
recover res validation : True
